Import `cup_detection` modules

In [1]:
from cup_detection.ml import training as T
from cup_detection.ml import cardboard as MLCB
from cup_detection.ml import cup as MLC
from cup_detection.eye_tracker import render_distances as RD

from cup_detection.io import image_sequence as IS

import time
import os
import pandas as pd

## Load pretrained classifiers. 

This function returns a dictionary with the keys `card-board` for the cardboard classifier and `cups` for the cups detection classifier.

In [3]:
t1 = time.time()
clfrs = T.load_all_classifiers("/Volumes/Seagate_Backup_Plus_Drive_black/datasets/cups/script_output/saved_classifiers")

Set filename of input video

In [5]:
video_filename = "/Volumes/Seagate Backup Plus Drive/datasets/cups/vt_encoded/CORIA/Scan Path_table (17)_table-23-recording/Scan Path_table (17)_table-23-recording_full.mp4"
video_folder = "/Volumes/Seagate_Backup_Plus_Drive_black/datasets/cups/image_sequences/CORIA/Scan Path_table (17)_table-23-recording"

In [ ]:
IS.video_to_image_sequence(video_file, video_folder)

## Classify each frame of the video as cardboard or not.

The result is a dictionary with the keys:

* `raw`: with the frame-wise classification results (boolean `np.array` with `1` for frames with cardboard and `0` for frames without)
* `smooth`: with the frame-wise classification results after applying the smoothing function which removes spurious noise in the classification result in `raw`

In [5]:
t2 = time.time()
card_board_frames = MLCB.get_card_board_frames_from_video(video_folder, clfrs['card-board'])

## Locate the blue cup in frames without cardboard
This function returns a pandas dataframe with the columns

* `Frame`: framenumber of event
* `cup x`: x coordinate
* `cup y`: y coordinate

with the argument `verbose=True` the function plots the current framenumber after each 100 frames the function iterated through

In [6]:
t3 = time.time()
cl_df = MLC.cup_predictions_outside_cardboard(video_folder, clfrs['cups'], 
                                              card_board_frames['smooth'],
                                              verbose=True)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400


Save cup locations if necessary

In [15]:
out_folder = "."
cl_df.to_csv(os.path.join(out_folder, "cup_locs.csv"))

In [2]:
out_folder = "."
cl_df = pd.read_csv(os.path.join(out_folder, "cup_locs.csv"))

## plot cup locations and eyetracker locations into frames

First load the eye-tracker data

In [3]:
et_filename = "/Volumes/Seagate_Backup_Plus_Drive_black/datasets/cups/events/CORIA/SWTS2 20150312 CORIA_table (17)_023_Trial001 Samples.txt"
et_df = RD.load_eye_tracker_data(et_filename)

call the plotting function with eye-tracker data and cup-locations

In [3]:
t4 = time.time()
out_folder = "/Volumes/Seagate_Backup_Plus_Drive_black/datasets/cups/script_output/frames_out_pipeline_test3"
RD.tracking_on_video(video_folder, et_df, cl_df, out_folder)

In [7]:
t5 = time.time()

### Render the plots into video

In [4]:
out_video = os.path.join(out_folder, "video.mp4")
IS.image_sequence_to_video(out_folder, out_video)

None


## Approximate timings for a 3:48min video

It takes about 

* 1h7min to detect card-board frames
* 13min to get cup locations
* 1h17min to plot the result on video frames

In [11]:
print "{} {} {} {}".format((t2 - t1) / 60, (t3 - t2) / 60, (t4 - t3) / 60, (t5- t4) / 60)

0.224729450544 87.6137160977 1118.02125085 148.668739518


## convert frames into video

In [12]:
from subprocess import call
call(["ffmpeg", "-r 24", "-i {}/frame_%04d.png".format(out_folder),
      "-r 24", "-pix_fmt yuv420p", "{}/video.mp4".format(out_folder)], shell=True)

1

In [10]:
call(["ls", "-l"])

0

In [11]:
call(["ffmpeg"])

1

In [13]:
import os
os.system("ffmpeg -r 24 -i {bf}/frame_%04d.png -r 24 -pix_fmt yuv420p {bf}/video.mp4".format(bf=out_folder))

256